In [ ]:
from math import pi,sin
angle = int(input('''enter the number for pressure angle : 1- 14.5 full depth
2- 20 full depth
3 - 20 stub tooth ''')) 
P = int(input("Power in KW")) # Power
N1 = int(input("Pinion speed in rpm")) # Pinion Speed
N2 = int(input("Gear speed in rpm")) # Gear Speed
center_distance =int(input("center distance if any "))
z1 =int(input("number of teeth in pinion")) # Pinion Tooth
z2 =int(input("number of teeth in gear")) # Gear Tooth
d1 =int(input("dia in pinion")) # Pinion Dia
d2 =int(input("dia of gear")) # Gear Dia
sigma_01=float(input("allowable stress of pinion")) # Pinion Stress
sigma_02 =float(input("allowable stress in gear")) # gear Stress
i =float(input("velocity ratio/transmission ratio")) # velocity ratio
teeth = z1

In [ ]:
def tangential_tooth_load():
  velocity_pinion_in_m=pi*z1*N1/60000
  velocity_gear_in_m=pi*z2*N2/60000
  weaker_member = weaker_mem(angle,z1, z2, sigma_01, sigma_02)
  if weaker_member:
      return 1000*P*service_factor/velocity_pinion_in_m
  else:
    teeth = z2
    return 1000*P*service_factor/velocity_gear_in_m
 
def dynamic_load():
  dynamic_f= induced_F + 20.67*velocity*(error*face_width + induced_F)/(20.67*velocity + (error*face_width + induced_F)**(1/2))
 
def s_fac(): 
  load_factor_k=sin(Angle)*(1/modulus_pinion + 1/modulus_gear)/1.4
  wear_load=pinion_dia*face_width*(2*gear_tooth/(pinion_tooth+gear_tooth))
  s_fac_val=dynamicload/(wear_load*load_factor_k)

Lewis eqn tangential load

In [ ]:
def lewis_tangential(sigma, y):
  Ft_lewis = sigma * 10 * y * pi
  return Ft_lewis


Pitch line velocity

In [ ]:
def pitch_line_vel(rpm, teeth):
  V_m = (pi * teeth * rpm)/60000
  return V_m


m3Kv

In [ ]:
def m3Kv(Ft_tangential, Ft_lewis):
  m3_Kv = Ft_tangential/Ft_lewis
  return  m3_Kv


In [ ]:
def module(V_m, m3_Kv):
  mod = [1, 1.25, 1.5, 2, 2.5, 3, 4, 5, 6, 8, 10, 12, 16, 20, 25, 32, 40, 50, 60, 80, 100]
  for m in mod:
    Vm = V_m * m
    if Vm <= 8:
      Cv = 3.05/(3.05 + Vm)
    elif Vm <= 13:
      Cv = 4.58/(4.58 + Vm)
    elif Vm <= 20:
      Cv = 6.1/(6.1 + Vm)
    else:
      Cv = 0.7625/(1.0167 + Vm)
    if (m ** 3) * Cv >= m3_Kv:
      break

  return m


In [ ]:
def check_for_stress(module, sigma, Cv, Ft_tangential, y):
  sigma_allowable = sigma * Cv
  sigma_induced = Ft_tangential/(module * 10 * module * y * pi * module)
  if sigma_allowable > sigma_induced:
    return True
  else:
    return False

In [ ]:
def weaker_mem(angle,pinion_tooth,gear_tooth,sigma01,sigma02):
 if angle == 1:
   y1 = 0.124 - 0.684/pinion_tooth
   y2 = 0.124 - 0.684/gear_tooth
 elif angle == 2:
   y1= 0.154-0.912/pinion_tooth
   y2= 0.154-0.912/gear_tooth
 elif angle == 3:
   y1 = 0.175 - .95/pinion_tooth
   y2 = 0.175 - .95/gear_tooth
  if sigma01*y1 < sigma02*y2:
   return True
 else:
   return False

In [ ]:
def velocity_factor(v):
 if v <= 8:
   return 3.05/(3.05+v)
 elif v<=13:
   return 4.58/(4.58+v)
 elif v <= 20:
   return 6.1/(6.1+v)
 else:
   return 0.7625/(1.0167+v)

In [8]:
import numpy as np
from numpy import sin
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import pickle
df = pd.read_csv('maxtootherror.csv')
tf = df.to_numpy()
tf[:,1]
x = tf[:,0]
y = tf[:,1]
poly_reg=PolynomialFeatures(degree=3)
X_poly=poly_reg.fit_transform(x)
poly_reg.fit(X_poly,y)
lin_reg2=LinearRegression()
lin_reg2.fit(X_poly,y)
filename = 'polynomial_preprocess.sav'
filename1 = 'linear_model.sav'
pickle.dump(poly_reg, open(filename,'wb'))
pickle.dump(lin_reg2, open(filename1, 'wb'))


 
def error_fac(v):
   pickle.load(open('polynomial_preprocess.sav','rb'))
   a = poly_reg.fit_transform(np.array([[v]]))
   return lin_reg2.predict(a)

print(error_fac(1.5)[0,0])

ValueError: Expected 2D array, got 1D array instead:
array=[ 1.25  2.5   3.75  5.    6.25  7.5   8.75 10.   11.25 12.5  13.75 15.
 16.25 17.5  20.   22.5  25.  ].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [23]:
import pickle
lin_reg2 = pickle.load(open('linear_model.sav','rb'))
poly_reg = pickle.load(open('polynomial_preprocess.sav','rb'))
x = poly_reg.fit_transform(np.array([[1.5]]))
lin_reg2.predict(x)

array([[0.0887978]])

In [9]:
import scipy.interpolate
y_interp = scipy.interpolate.interp1d(x, y)
print(y_interp(1.5))

0.09


In [13]:
i = [1,1.5]
j = [4,7]
j_interp = scipy.interpolate.interp1d(i,j)
j_interp(1.25)

array(5.5)

In [31]:
er = pd.read_csv('Deformationfactor.csv')
t = er.query('Material1 == 1').query('Toothform == 2')
t1 =[c for c in er.columns]

t1 = [float(i) for i in t1[2:]]

def error1(z,a,v):
    er = pd.read_csv('Deformationfactor.csv')
    t1 =[c for c in er.columns]
    t1 = [float(i) for i in t1[2:]]
    t = er.query(f'Material1 == {z}').query(f'Toothform == {a}').to_numpy()
    e = error_fac(v)
    y = t[0,2:]
    C_interp = scipy.interpolate.interp1d(t1,y)
    C = C_interp(e)
    return C

error1(2,1,3.5)


array([[549.63005261]])

In [ ]:
rt  = pd.read_csv('Untitled.csv')
rt[rt.iloc[:,4] >=630].iloc[0,:]

In [33]:
from math import pi,sin

pi*14.5/180

0.2530727415391778

In [34]:
sin(pi*30/180)

0.49999999999999994

In [3]:
import pandas as pd
bhntable = pd.read_csv('S_fac&BHN.csv')
t = bhntable[bhntable['s_fac'] >= 620].iloc[0,:]


In [4]:
import pandas as pd
import numpy as np
er = pd.read_csv('Deformationfactor.csv')
t1 =[c for c in er.columns]
t1 = [float(i) for i in t1[2:]]
t = er.query(f'Material1 == 3')
print(t)
t= t.query('Toothform == 2')
print(t)
y = t[0,2:]
y

   Material1  Toothform  0.01  0.02  0.04  0.06  0.08
2          3          1   110   220   440   660   880
8          3          3   119   238   476   714   952
Empty DataFrame
Columns: [Material1, Toothform, 0.01, 0.02, 0.04, 0.06, 0.08]
Index: []


TypeError: '(0, slice(2, None, None))' is an invalid key